In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt



postulantesEducacion = pd.read_csv('resources/tar/fiuba_1_postulantes_educacion.csv')
postulantesGenero = pd.read_csv('resources/tar/fiuba_2_postulantes_genero_y_edad.csv')
vistas = pd.read_csv('resources/tar/fiuba_3_vistas.csv')
postulaciones = pd.read_csv('resources/tar/fiuba_4_postulaciones.csv')
avisosOnline = pd.read_csv('resources/tar/fiuba_5_avisos_online.csv')
avisosDetalle = pd.read_csv('resources/tar/fiuba_6_avisos_detalle.csv')

#cargamos zips
avisosDetalleZIP = pd.read_csv('resources/zip/fiuba_6_avisos_detalle.csv')
avisosDetalleMissing = pd.read_csv('resources/zip/fiuba_6_avisos_missing.csv')

postulantesEducacionZip = pd.read_csv('resources/zip/fiuba_1_postulantes_educacion.csv')
postulantesGeneroZip = pd.read_csv('resources/zip/fiuba_2_postulantes_genero_y_edad.csv')

vistasZIP = pd.read_csv('resources/zip/fiuba_3_vistas.csv')

In [2]:
testingSet = pd.read_csv('resources/test_final_100k.csv')

#### Para mi prueba de algoritmo de Machine Learning, primero preparo algunos datos y genero un dataset para probar y evaluar mis resultados.

# Transformamos Categorias en Numerical

## PostulantesGenero

In [2]:
postulantesGenero.head()

,idpostulante,fechanacimiento,sexo
0,6MM,1985-01-01,MASC
1,Nzz,NaN,NO_DECLARA
2,ZX1,NaN,NO_DECLARA
3,Nq5,NaN,NO_DECLARA
4,ebE,1952-07-07,MASC


## Transformamos fecha de nacimiento en edad

In [3]:
import datetime as dt

postulantesGenero['fechanacimiento'] = pd.to_datetime(postulantesGenero['fechanacimiento'], format='%Y-%m-%d', errors='coerce')
postulantesGenero = postulantesGenero.dropna(subset=['fechanacimiento'])

hoy = pd.Timestamp(dt.datetime.now())
postulantesGenero['edad'] = (hoy - postulantesGenero['fechanacimiento']).astype('<m8[Y]')

In [4]:
postulantesGenero[postulantesGenero['idpostulante'] == 'PmqM3oj']  #caso raro

,idpostulante,fechanacimiento,sexo,edad
137797,PmqM3oj,1982-12-15,0.0,35.0


In [5]:
postulantesGeneroOHC = pd.concat([postulantesGenero,pd.get_dummies(postulantesGenero['sexo'], prefix='sexo')],axis=1)

In [6]:
postulantesGeneroOHC = postulantesGeneroOHC[postulantesGeneroOHC['sexo_0.0'] == 0]
postulantesGeneroOHC = postulantesGeneroOHC[['idpostulante','fechanacimiento','sexo_FEM','sexo_MASC','sexo_NO_DECLARA','edad']]

In [91]:
postulantesGeneroOHC.head()

,idpostulante,fechanacimiento,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,edad
0,6MM,1985-01-01,0,1,0,33.0
4,ebE,1952-07-07,0,1,0,65.0
7,NAjM,1962-06-09,1,0,0,56.0
10,ZjlZ,1970-01-25,0,1,0,48.0
11,8wPl,1973-03-13,0,1,0,45.0


## Avisos Detalle

In [11]:
avisosDetalle.head()

,idaviso,idpais,titulo,descripcion,nombre_zona,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,1111556097,1,Responsable de Marketing-Producto Div. Constru...,<p>Buscamos un <strong>Responsable de Producto...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Desarrollo de Negocios,Sinteplast S.A.
1,1111949392,1,Asesor de Servicios - Postventa Chevrolet,"<p><span style="""">Para nuestro cliente, conces...",Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Ingeniería Eléctrica y Electrónica,Roycan
2,1112145935,1,Analistas de Impuestos Sr Ssr y Jr,<p>Importante Estudio de Auditoria y Consultor...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Impuestos,Lighthouse Smart
3,1112146010,1,Software Engineer,<p> </p><p>Chevron is a multinational company ...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Programación,Chevron Buenos Aires Shared Services Center
4,1112211475,1,Business Developer Manager,"<p>Importante empresa de energías renovables, ...",Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Comercial,RANDSTAD


#### Convierto datos string en numericals

In [93]:
#Para losrimeros casos, me voy a quedar con la zona, tipo de trabajo, nivel laboral y nombre de area
avisosDetalleFiltered = avisosDetalle[['idaviso','nombre_zona','tipo_de_trabajo','nivel_laboral']]

In [95]:
avisosDetalleFiltered = avisosDetalleFiltered.loc[avisosDetalleFiltered['nombre_zona'].notnull(),:]
avisosDetalleFiltered = avisosDetalleFiltered.loc[avisosDetalleFiltered['tipo_de_trabajo'].notnull(),:]
avisosDetalleFiltered = avisosDetalleFiltered.loc[avisosDetalleFiltered['nivel_laboral'].notnull(),:]

In [96]:
avisosDetalleOHC = pd.concat([avisosDetalleFiltered,pd.get_dummies(avisosDetalleFiltered['nombre_zona'], prefix='zona')],axis=1)
avisosDetalleOHC = pd.concat([avisosDetalleOHC,pd.get_dummies(avisosDetalleFiltered['tipo_de_trabajo'], prefix='tipo')],axis=1)
avisosDetalleOHC = pd.concat([avisosDetalleOHC,pd.get_dummies(avisosDetalleFiltered['nivel_laboral'], prefix='nivel')],axis=1)

In [97]:
avisosDetalleOHC.drop(['nombre_zona'],axis=1, inplace=True)
avisosDetalleOHC.drop(['tipo_de_trabajo'],axis=1, inplace=True)
avisosDetalleOHC.drop(['nivel_laboral'],axis=1, inplace=True)

In [98]:
avisosDetalleOHC.head()

,idaviso,zona_Capital Federal,zona_GBA Oeste,zona_Gran Buenos Aires,tipo_Fines de Semana,tipo_Full-time,tipo_Part-time,tipo_Pasantia,tipo_Por Contrato,tipo_Por Horas,tipo_Primer empleo,tipo_Teletrabajo,tipo_Temporario,tipo_Voluntario,nivel_Gerencia / Alta Gerencia / Dirección,nivel_Jefe / Supervisor / Responsable,nivel_Junior,nivel_Otro,nivel_Senior / Semi-Senior
0,1111556097,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
1,1111949392,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
2,1112145935,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
3,1112146010,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
4,1112211475,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1


In [99]:
postulacionesFiltered = postulaciones[['idpostulante','idaviso']]
postulacionesFiltered.count()

idpostulante    4909641
idaviso         4909641
dtype: int64

In [100]:
postulacionesFiltered =  postulacionesFiltered.drop_duplicates()
postulacionesFiltered.count()

idpostulante    4909025
idaviso         4909025
dtype: int64

In [103]:
postulacionesSample = postulacionesFiltered.sample(frac=0.0005)
postulacionesSample.count()

idpostulante    2455
idaviso         2455
dtype: int64

In [112]:
todoMergeado = pd.merge(postulacionesSample,avisosDetalleOHC,how='inner',left_on='idaviso',right_on='idaviso')
todoMergeado = pd.merge(todoMergeado,postulantesGeneroOHC, how='inner', left_on='idpostulante',right_on='idpostulante')

In [113]:
todoMergeado['sepostulo'] = 1

In [114]:
todoMergeado.head()

,idpostulante,idaviso,zona_Capital Federal,zona_GBA Oeste,zona_Gran Buenos Aires,tipo_Fines de Semana,tipo_Full-time,tipo_Part-time,tipo_Pasantia,tipo_Por Contrato,...,nivel_Jefe / Supervisor / Responsable,nivel_Junior,nivel_Otro,nivel_Senior / Semi-Senior,fechanacimiento,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,edad,sepostulo
0,Bm5l6PR,1112410160,0,0,1,0,1,0,0,0,...,1,0,0,0,1995-07-16,1,0,0,22.0,1
1,NVL1x5,1112326623,1,0,0,0,1,0,0,0,...,0,0,0,1,1987-10-29,1,0,0,30.0,1
2,rm09eOe,1112296005,0,0,1,0,1,0,0,0,...,0,0,0,1,1988-12-16,0,1,0,29.0,1
3,rmKX8KA,1112296005,0,0,1,0,1,0,0,0,...,0,0,0,1,1983-01-17,0,1,0,35.0,1
4,zv82BW8,1112296005,0,0,1,0,1,0,0,0,...,0,0,0,1,1994-03-07,0,1,0,24.0,1


In [115]:
todoMergeado.drop('fechanacimiento',axis=1,inplace=True)

2439

In [124]:
len(todoMergeado)

2207

# Armamos avisos para los que no se postulo

In [125]:
nopostulados = pd.DataFrame()
for i in range(0,len(todoMergeado.head(10)['idaviso'])):
    rowAviso = todoMergeado.iloc[i:i+1]
    postuRandom= postulaciones.sample(frac=0.00005)[['idpostulante']]
    postuRandom['idaviso'] =rowAviso.iloc[0]['idaviso'];
    nopostulados = pd.concat([nopostulados,postuRandom])
    
nopostulados = nopostulados.drop_duplicates()
len(nopostulados)    

2435

In [ ]:
##Avisos para los que no se postulo

##avisosNoPostulado = pd.merge(postulaciones,todoMergeado,on=['idpostulante', 'idaviso'] ,how='outer')
##avisosNoPostulado = pd.merge(postulacionesSample,todoMergeadoSample,on=['idpostulante', 'idaviso'] ,how='outer')

In [132]:
#avisosNoPostulado = pd.merge(postulacionesSample,todoMergeadoSample,on=['idpostulante', 'idaviso'] ,how='outer')
#avisosNoPostulado = avisosNoPostulado[avisosNoPostulado['fechapostulacion'].isnull()]

avisosNoPostulado= nopostulados;

In [133]:
avisosNoPostulado['sepostulo'] = 0

In [134]:
avisosNoPostulado = pd.merge(nopostulados,avisosDetalleOHC,how='inner',left_on='idaviso',right_on='idaviso')
avisosNoPostulado = pd.merge(avisosNoPostulado,postulantesGeneroOHC, how='inner', left_on='idpostulante',right_on='idpostulante')

In [135]:
#todoMergeadoFINAL = pd.concat(avisosNoPostulado,todoMergeado, ignore_index=True)
todoMergeadoFINAL = pd.concat([avisosNoPostulado, todoMergeado])

In [93]:
#Revisar por que no se borran
#todoMergeadoFINAL.drop(['tipo_de_trabajo'],axis=1, inplace=True)
#todoMergeadoFINAL = todoMergeadoFINAL[['idpostulante','fechanacimiento','sexo_FEM','sexo_MASC','sexo_NO_DECLARA']]
#todoMergeadoFINAL.drop('sexo',axis=1, inplace=True)

#todoMergeadoFINAL.drop(['nombre_zona'],axis=1, inplace=True)
#todoMergeadoFINAL.drop(['nivel_laboral'],axis=1, inplace=True)
#todoMergeadoFINAL.drop(['nombre_area'],axis=1, inplace=True)

In [138]:
#lo separo porque sino explota todo
datosForTraining = todoMergeadoFINAL

In [139]:
datosForTraining.head()

,edad,idaviso,idpostulante,nivel_Gerencia / Alta Gerencia / Dirección,nivel_Jefe / Supervisor / Responsable,nivel_Junior,nivel_Otro,nivel_Senior / Semi-Senior,sepostulo,sexo_FEM,...,tipo_Pasantia,tipo_Por Contrato,tipo_Por Horas,tipo_Primer empleo,tipo_Teletrabajo,tipo_Temporario,tipo_Voluntario,zona_Capital Federal,zona_GBA Oeste,zona_Gran Buenos Aires
0,26.0,1112410160,qekrW81,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
1,29.0,1112410160,pzjDkN3,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
2,29.0,1112410160,bOzqo29,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
3,47.0,1112410160,1avP3N,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,30.0,1112410160,lDLV6Oz,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1


In [141]:
datosForTrainingNumericals = datosForTraining.drop(['fechanacimiento','fechapostulacion'],axis=1)

In [142]:
datosForTrainingNumericals.dropna(axis=0,inplace=True, how='any')

In [143]:
datosForTrainingNumericals.dtypes

area_Abastecimiento                           float64
area_Administracion de Seguros                float64
area_Administración                           float64
area_Administración de Base de Datos          float64
area_Administración de Personal               float64
area_Almacén / Depósito / Expedición          float64
area_Análisis Funcional                       float64
area_Análisis de Riesgos                      float64
area_Apoderado Aduanal                        float64
area_Arquitectura                             float64
area_Arte y Cultura                           float64
area_Asesoría Legal Internacional             float64
area_Asistente                                float64
area_Asistente de Tráfico                     float64
area_Atención al Cliente                      float64
area_Auditoría                                float64
area_Auditoría Médica                         float64
area_Back Office                              float64
area_Bienestar Estudiantil  

In [144]:
datosForTrainingNumericals.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 72322 entries, 0 to 72321
Columns: 220 entries, area_Abastecimiento to zona_Gran Buenos Aires
dtypes: float64(219), int64(1)
memory usage: 121.9 MB


In [145]:
datosForTrainingNumericals.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 72322 entries, 0 to 72321
Columns: 220 entries, area_Abastecimiento to zona_Gran Buenos Aires
dtypes: float64(219), int64(1)
memory usage: 121.9 MB


In [148]:
datosForTrainingNumericals = datosForTraining.drop('idaviso',axis=1)
datosForTrainingNumericalsSinPostulacion  = datosForTrainingNumericals.drop('sepostulo',axis=1)
datosForTrainingNumericals = datosForTrainingNumericals.drop('idpostulante',axis=1)
datosForTrainingNumericalsSinPostulacion = datosForTrainingNumericalsSinPostulacion.drop('idpostulante',axis=1)

In [149]:
 mylist = list(datosForTrainingNumericals.select_dtypes(include=['object']).columns)

# Entrenamos Machine Learning

In [150]:
import pandas
from sklearn import model_selection
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import RadiusNeighborsRegressor
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn import linear_model
from sklearn import svm
from sklearn import tree
from sklearn import gaussian_process
from sklearn import neural_network
from sklearn.linear_model import LinearRegression, LogisticRegression



#Pre-procesamiento
from sklearn import preprocessing

In [151]:
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(datosForTrainingNumericalsSinPostulacion, datosForTrainingNumericals['sepostulo'], test_size=0.2, random_state=7)

In [155]:
models = []
models.append(('RFR10', RandomForestRegressor(n_estimators=10, max_features='sqrt')))
models.append(('RFR20', RandomForestRegressor(n_estimators=20, max_features='sqrt')))
models.append(('RFR50',  RandomForestRegressor(n_estimators=50, max_features='sqrt')))
models.append(('RFR60',  RandomForestRegressor(n_estimators=60, max_features='sqrt')))
models.append(('RFR100',  RandomForestRegressor(n_estimators=100, max_features='sqrt')))

models.append(('RFC10', RandomForestRegressor(n_estimators=10, max_features='sqrt')))
models.append(('RFC20', RandomForestRegressor(n_estimators=20, max_features='sqrt')))
models.append(('RFC50',  RandomForestRegressor(n_estimators=50, max_features='sqrt')))
models.append(('RFC60',  RandomForestRegressor(n_estimators=60, max_features='sqrt')))
models.append(('RFC100',  RandomForestClassifier(n_estimators=100, max_features='sqrt')))

models.append(('KNN 2', KNeighborsClassifier(n_neighbors=2)))
models.append(('KNN 3', KNeighborsClassifier(n_neighbors=3)))
models.append(('LR', linear_model.LinearRegression()))
models.append(('RIDGE.05', linear_model.Ridge (alpha = .5)))
models.append(('LASO.01', linear_model.Ridge (alpha = .1)))
models.append(('LASOLARS.05',linear_model.LassoLars (alpha = 0.5)))
models.append(('BAYESIAN',linear_model.BayesianRidge()))
models.append(('SVR',svm.LinearSVR()))
models.append(('DES-TREE',tree.DecisionTreeRegressor()))
models.append(('NB', GaussianNB()))
models.append(('LDA', LinearDiscriminantAnalysis()))
 
TestModels = pd.DataFrame()
tmp = {}

results = []
names = []
for name, model in models:
    kfold = model_selection.KFold(n_splits=10, random_state=7)
    cv_results = model_selection.cross_val_score(model, X_train,Y_train)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

RFR10: 0.081268 (0.005587)
RFR20: 0.083233 (0.004481)
RFR50: 0.089590 (0.007050)
RFR60: 0.091051 (0.004208)
RFR100: 0.090027 (0.005576)
RFC10: 0.082198 (0.007145)
RFC20: 0.090133 (0.005974)
RFC50: 0.091368 (0.004178)
RFC60: 0.089136 (0.004179)
RFC100: 0.609500 (0.002733)
KNN 2: 0.596742 (0.002619)
KNN 3: 0.589947 (0.016717)
LR: 0.144936 (0.016587)
RIDGE.05: 0.145063 (0.016453)
LASO.01: 0.144984 (0.016571)
LASOLARS.05: -0.002537 (0.001803)
BAYESIAN: 0.144456 (0.015281)
SVR: -0.610361 (0.377650)
DES-TREE: 0.061187 (0.006413)
NB: 0.632290 (0.004506)
LDA: 0.630385 (0.011350)


C:\Users\nicol\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\nicol\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\nicol\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


In [158]:
from sklearn.model_selection import cross_val_score

RFCModel = RandomForestClassifier(n_estimators=100, max_features='sqrt')
scoresRFC = cross_val_score(RFCModel, datosForTrainingNumericalsSinPostulacion, datosForTrainingNumericals['sepostulo'], cv=5)
print("---RFC----")
print(scoresRFC)


GaussianNBModel = GaussianNB()
scoresRFR = cross_val_score(GaussianNBModel, datosForTrainingNumericalsSinPostulacion, datosForTrainingNumericals['sepostulo'], cv=5)
print("---GAUSS----")
print(scoresRFR)


DTModel = LinearDiscriminantAnalysis()
scoresDT = cross_val_score(DTModel, datosForTrainingNumericalsSinPostulacion, datosForTrainingNumericals['sepostulo'], cv=5)
print("---LDA----")
print(scoresDT)

---RFC----
[ 0.26030369  0.55639913  0.5320304   0.52334419  0.22692725]
---GAUSS----
[ 0.1670282   0.66160521  0.61237785  0.62540717  0.61780673]
---LDA----
[ 0.17353579  0.59327549  0.56026059  0.56677524  0.2019544 ]


C:\Users\nicol\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\nicol\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\nicol\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\nicol\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\nicol\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


### Segundo Test

In [159]:
#Aumento la cantidad de postulacionesSample
postulacionesSample = postulacionesFiltered.sample(frac=0.05)
postulacionesSample.count()

idpostulante    245451
idaviso         245451
dtype: int64

In [166]:
todoMergeado = pd.merge(postulacionesSample,avisosDetalleOHC,how='inner',left_on='idaviso',right_on='idaviso')
todoMergeado = pd.merge(todoMergeado,postulantesGeneroOHC, how='inner', left_on='idpostulante',right_on='idpostulante')
todoMergeado['sepostulo'] = 1
todoMergeado.drop('fechanacimiento',axis=1,inplace=True)

In [167]:
nopostulados = pd.DataFrame()
for i in range(0,len(todoMergeado.head(20)['idaviso'])):
    rowAviso = todoMergeado.iloc[i:i+1]
    postuRandom= postulaciones.sample(frac=0.003)[['idpostulante']]
    postuRandom['idaviso'] =rowAviso.iloc[0]['idaviso'];
    nopostulados = pd.concat([nopostulados,postuRandom])
    
nopostulados = nopostulados.drop_duplicates()
len(nopostulados)  

avisosNoPostulado= nopostulados;
avisosNoPostulado['sepostulo'] = 0

In [168]:
avisosNoPostulado = pd.merge(nopostulados,avisosDetalleOHC,how='inner',left_on='idaviso',right_on='idaviso')
avisosNoPostulado = pd.merge(avisosNoPostulado,postulantesGeneroOHC, how='inner', left_on='idpostulante',right_on='idpostulante')

In [187]:
todoMergeadoFINAL = pd.concat([avisosNoPostulado, todoMergeado])
datosForTraining = todoMergeadoFINAL

datosForTraining.drop('fechanacimiento',axis=1,inplace=True)

datosForTrainingNumericals = datosForTraining
datosForTrainingNumericals.dropna(axis=0,inplace=True, how='any')

In [188]:
datosForTrainingNumericals = datosForTraining.drop('idaviso',axis=1)
datosForTrainingNumericalsSinPostulacion  = datosForTrainingNumericals.drop('sepostulo',axis=1)
datosForTrainingNumericals = datosForTrainingNumericals.drop('idpostulante',axis=1)
datosForTrainingNumericalsSinPostulacion = datosForTrainingNumericalsSinPostulacion.drop('idpostulante',axis=1)

In [189]:
datosForTrainingNumericals.columns

Index(['edad', 'nivel_Gerencia / Alta Gerencia / Dirección',
       'nivel_Jefe / Supervisor / Responsable', 'nivel_Junior', 'nivel_Otro',
       'nivel_Senior / Semi-Senior', 'sepostulo', 'sexo_FEM', 'sexo_MASC',
       'sexo_NO_DECLARA', 'tipo_Fines de Semana', 'tipo_Full-time',
       'tipo_Part-time', 'tipo_Pasantia', 'tipo_Por Contrato',
       'tipo_Por Horas', 'tipo_Primer empleo', 'tipo_Teletrabajo',
       'tipo_Temporario', 'tipo_Voluntario', 'zona_Capital Federal',
       'zona_GBA Oeste', 'zona_Gran Buenos Aires'],
      dtype='object')

In [190]:
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(datosForTrainingNumericalsSinPostulacion, datosForTrainingNumericals['sepostulo'], test_size=0.2, random_state=7)

In [191]:
models = []
models.append(('RFR10', RandomForestRegressor(n_estimators=10, max_features='sqrt')))
models.append(('RFR20', RandomForestRegressor(n_estimators=20, max_features='sqrt')))
models.append(('RFR50',  RandomForestRegressor(n_estimators=50, max_features='sqrt')))
models.append(('RFR60',  RandomForestRegressor(n_estimators=60, max_features='sqrt')))
models.append(('RFR100',  RandomForestRegressor(n_estimators=100, max_features='sqrt')))

models.append(('RFC10', RandomForestRegressor(n_estimators=10, max_features='sqrt')))
models.append(('RFC20', RandomForestRegressor(n_estimators=20, max_features='sqrt')))
models.append(('RFC50',  RandomForestRegressor(n_estimators=50, max_features='sqrt')))
models.append(('RFC60',  RandomForestRegressor(n_estimators=60, max_features='sqrt')))
models.append(('RFC100',  RandomForestClassifier(n_estimators=100, max_features='sqrt')))

models.append(('KNN 2', KNeighborsClassifier(n_neighbors=2)))
models.append(('KNN 3', KNeighborsClassifier(n_neighbors=3)))
models.append(('LR', linear_model.LinearRegression()))
models.append(('RIDGE.05', linear_model.Ridge (alpha = .5)))
models.append(('LASO.01', linear_model.Ridge (alpha = .1)))
models.append(('LASOLARS.05',linear_model.LassoLars (alpha = 0.5)))
models.append(('BAYESIAN',linear_model.BayesianRidge()))
models.append(('SVR',svm.LinearSVR()))
models.append(('DES-TREE',tree.DecisionTreeRegressor()))
models.append(('NB', GaussianNB()))
models.append(('LDA', LinearDiscriminantAnalysis()))
 
TestModels = pd.DataFrame()
tmp = {}

results = []
names = []
for name, model in models:
    kfold = model_selection.KFold(n_splits=10, random_state=7)
    cv_results = model_selection.cross_val_score(model, X_train,Y_train)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

RFR10: 0.353495 (0.003226)
RFR20: 0.353552 (0.003170)
RFR50: 0.353517 (0.003224)
RFR60: 0.353573 (0.003202)
RFR100: 0.353556 (0.003218)
RFC10: 0.353473 (0.003255)
RFC20: 0.353491 (0.003256)
RFC50: 0.353580 (0.003195)
RFC60: 0.353543 (0.003205)
RFC100: 0.757973 (0.001559)
KNN 2: 0.666637 (0.010344)
KNN 3: 0.698880 (0.004891)
LR: 0.274874 (0.002613)
RIDGE.05: 0.274874 (0.002613)
LASO.01: 0.274874 (0.002613)
LASOLARS.05: -0.000004 (0.000003)
BAYESIAN: 0.274873 (0.002613)
SVR: -0.257657 (0.633758)
DES-TREE: 0.353423 (0.003229)
NB: 0.577928 (0.000302)


C:\Users\nicol\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\nicol\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


LDA: 0.734154 (0.001160)


C:\Users\nicol\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


In [192]:
from sklearn.model_selection import cross_val_score

RFCModel = RandomForestClassifier(n_estimators=100, max_features='sqrt')
scoresRFC = cross_val_score(RFCModel, datosForTrainingNumericalsSinPostulacion, datosForTrainingNumericals['sepostulo'], cv=5)
print("---RFC----")
print(scoresRFC)


DTModel = LinearDiscriminantAnalysis()
scoresDT = cross_val_score(DTModel, datosForTrainingNumericalsSinPostulacion, datosForTrainingNumericals['sepostulo'], cv=5)
print("---LDA----")
print(scoresDT)

---RFC----
[ 0.73955803  0.77456053  0.79105053  0.78274581  0.69454762]


C:\Users\nicol\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\nicol\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\nicol\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\nicol\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


---LDA----
[ 0.71511166  0.74360801  0.76693064  0.76833228  0.67665335]


C:\Users\nicol\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


# Prueba 1 con set de test

In [95]:
testingSet = pd.read_csv('resources/test_final_100k.csv')

In [167]:
#testingSet.head()

testingSetMerged = pd.merge(testingSet,postulantesEducacionOHC,how='left',on='idpostulante')
testingSetMerged = pd.merge(testingSetMerged,postulantesGeneroOHC,how='left',on='idpostulante')

In [168]:
testingSetMerged = pd.merge(testingSetMerged,avisosDetalleOHC,how='left',on='idaviso')

In [169]:
#todoMergeado.columns
testingSetMerged = testingSetMerged.loc[testingSetMerged['id'].notnull(),:]

In [170]:
testingSetMerged = testingSetMerged.loc[testingSetMerged['id'].notnull(),:]
testingSetMerged = testingSetMerged.drop_duplicates(subset='id')

In [156]:
testingSetMerged.info()

NameError: name 'testingSetMerged' is not defined

In [171]:
testingSetMerged['test_set'] =1

In [172]:
datosForTrainingNumericals['train_set'] = 1

In [173]:
concatenados = pd.concat([testingSetMerged,datosForTrainingNumericals],axis=1)

In [174]:
setDatosToPredict = concatenados[concatenados['test_set'] == 1]

In [230]:
setDatosToPredict.info

<bound method DataFrame.info of              id       idaviso idpostulante  estado_Abandonado  \
0           0.0  7.392600e+05       6M9ZQR                0.0   
2           1.0  7.392600e+05       6v1xdL                0.0   
5           2.0  7.392600e+05       ezRKm9                0.0   
6           3.0  7.585800e+05       1Q35ej                0.0   
8           4.0  7.585800e+05       EAN4J6                0.0   
11          5.0  7.585800e+05       8R6pzR                0.0   
13          6.0  7.764200e+05       aZJ2XN                0.0   
14          7.0  7.764200e+05       Nmpo3J                0.0   
16          8.0  7.764200e+05       eVqWar                0.0   
17          9.0  8.208500e+05       6ZBD33                0.0   
19         10.0  8.208500e+05       1j4RNj                0.0   
20         11.0  8.208500e+05        ZOwE8                0.0   
22         12.0  8.208500e+05       NG4BaD                0.0   
23         13.0  9.148800e+05       6q6eNl                

In [175]:
setToTrain = concatenados[concatenados['train_set'] == 1]

In [176]:
setToTrain['edad'] = (hoy - setToTrain['fechanacimiento']).astype('<m8[Y]')

C:\Users\nicol\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [177]:
setToTrain = setToTrain.drop(['fechapostulacion','fechanacimiento','idaviso','idpostulante','id','test_set','train_set'],axis=1)

In [178]:
setToTrain.dropna(axis=0,inplace=True, how='any')

In [179]:
setToTrainNumericals = setToTrain.drop('sepostulo',axis=1)

In [182]:
setToTrainNumericals = setToTrain.drop('denominacion_empresa',axis=1)

In [183]:
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(setToTrainNumericals, setToTrain['sepostulo'], test_size=0.3, random_state=7)

In [184]:
g = GaussianNB()
g.fit(X_train, Y_train)

GaussianNB(priors=None)

In [185]:
setDatosToPredict['edad'] = (hoy - setDatosToPredict['fechanacimiento']).astype('<m8[Y]')

C:\Users\nicol\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [186]:
limpiado = setDatosToPredict.drop(['fechapostulacion','fechanacimiento','idaviso','idpostulante','id','test_set','train_set'],axis=1)

In [190]:
limpiado = limpiado.drop('sepostulo',axis=1)
limpiado = limpiado.drop('denominacion_empresa',axis=1)

In [191]:
from sklearn.preprocessing import Imputer
imp = Imputer(missing_values='NaN', strategy='most_frequent', axis=0)

In [195]:
limpiado = limpiado.loc[:,~limpiado.columns.duplicated()]

In [158]:
archivo2 = pd.DataFrame()
archivo2['id'] = testingSet.loc[:,['id']]
archivo2['sepostulo'] = prueba2
archivo2.to_csv('prueba2.csv', index=False)

In [100]:
testingSet.size

300000

# Prueba 2

In [162]:
rf =  RandomForestRegressor(n_estimators=100, max_features='sqrt')
rf.fit(X_train, Y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='sqrt', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [163]:
prueba3 =g.predict(imp.fit_transform(limpiado))

In [165]:
archivo2 = pd.DataFrame()
archivo2['id'] = testingSet.loc[:,['id']]
archivo2['sepostulo'] = prueba3
archivo2.to_csv('prueba3.csv', index=False)

In [ ]:
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(setToTrainNumericals, setToTrain['sepostulo'], test_size=0.3, random_state=7)